In [1]:
import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [2]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [3]:
url = "https://www.tripadvisor.com/Hotel_Review-g304554-d472105-Reviews-Trident_Hotel-Mumbai_Maharashtra.html"

In [4]:
main_review_list = list()
review_link = list()
name_list = list()
profile_list = list()
review_list = list()
date_list = list()
rating_list = list()
gender = list()

In [11]:
def get_data(doc):
    review = doc.find_all(class_ = "Qwuub")
    name = doc.find_all(class_ = "ui_header_link uyyBf")        
    main_review = doc.find_all(class_ = "QewHA H4 _a")
    date = doc.find_all(class_ = "teHYY _R Me S4 H3")
    rating = doc.find_all(class_ = "Hlmiy F1")
    for l in range(len(review)-1):
        main_review_list.append(review[l].get_text())
        review_link.append("https://www.tripadvisor.in/"+review[l].get('href'))
        n = name[l].get_text()
        p = requests.get("https://api.genderize.io/?name="+n.split(' ')[0]).text
        gender.append(p[p.find('gender')+9:p.find('probability')-3])
        name_list.append(n)
        profile_list.append("https://www.tripadvisor.in/"+name[l].get('href'))
        review_list.append(main_review[l].get_text())
        date_list.append(str(date[l])[str(date[l]).find("</span> ")+8:-7])
        rating_list.append(str(rating[l])[-17])

In [12]:
result = get_source(url).text
doc = BeautifulSoup(result, "html.parser")
get_data(doc)
print(len(name_list))

9


In [ ]:
for i in range(10,5000,10):
    page = get_source(url[:url.rfind('Reviews')+8]+'or'+str(i)+url[url.rfind('Reviews')+7:])
    print(page)
    result = page.text
    doc = BeautifulSoup(result, "html.parser")
    get_data(doc)
    print(len(name_list))

In [13]:
df = pd.DataFrame(list(zip(name_list, gender, profile_list, main_review_list, review_link, review_list, date_list, rating_list)),columns =['Name', 'Gender', 'Profile url','Review','Review url','Detailed Review','Stay Date','Rating'])
df

,Name,Gender,Profile url,Review,Review url,Detailed Review,Stay Date,Rating
0,Saubhik Datta,male,https://www.tripadvisor.in//Profile/dsaubhik2001,"Amazing property, exceptional service & fabulo...",https://www.tripadvisor.in//ShowUserReviews-g3...,Stayed at Trident from 2nd July 2022 for 5 nig...,July 2022,5
1,Wellness Doctor,male,https://www.tripadvisor.in//Profile/havepeaceo...,"Hotel Trident & Frangipani Restaurant, Nariman...",https://www.tripadvisor.in//ShowUserReviews-g3...,"Hi, this is Dr. Divya. We stayed at Hotel Trid...",July 2022,5
2,HarshadDASWANI,ul,https://www.tripadvisor.in//Profile/HarshadDAS...,Eternal Favourite & Perfect,https://www.tripadvisor.in//ShowUserReviews-g3...,This is a review for TRIDENT - NARIMAN POINT (...,June 2022,5
3,Dhanpat Parekh,male,https://www.tripadvisor.in//Profile/Dhanpar,Housekeeping,https://www.tripadvisor.in//ShowUserReviews-g3...,Room no 3125 Mr. Parekh Dhanpat House keepi...,July 2022,5
4,Dr. Jasem,ul,https://www.tripadvisor.in//Profile/Jmyousef,Dr. Jasem,https://www.tripadvisor.in//ShowUserReviews-g3...,Thank you all well done we felt like we are in...,July 2022,5
5,Navin John,male,https://www.tripadvisor.in//Profile/navinjohn1,Amazing Property - Exceptional Care!,https://www.tripadvisor.in//ShowUserReviews-g3...,"I give the Trident Hotel, Nariman Point my hig...",July 2022,5
6,Ravi M,male,https://www.tripadvisor.in//Profile/274ravim,Fabulous stay,https://www.tripadvisor.in//ShowUserReviews-g3...,For last two years I was staying in their sist...,June 2022,5
7,Nikhilesh,male,https://www.tripadvisor.in//Profile/899nikki,"Aneesh , Wayne and team. \rExceptional service 👏🏻",https://www.tripadvisor.in//ShowUserReviews-g3...,Me and my fiance recently dined at Frangipani ...,July 2022,5
8,AmitSinha1975,ul,https://www.tripadvisor.in//Profile/AmitSinha1975,Fabulous location,https://www.tripadvisor.in//ShowUserReviews-g3...,The hotel is located at one of the best places...,June 2022,3


In [ ]:
df.to_csv('Trident_Hotel-Mumbai.csv')